In [83]:
import sqlite3
import pandas as pd
from sqlite3 import Error

def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

def run_query(conn, query):
    cur = conn.cursor()
    cur.execute(query)
    results = cur.fetchall()
    return results

def convert_db_table_to_DF(conn, table):
    # get the names of the attributes in the database table
    header_query = "SELECT name FROM pragma_table_info('" + table + "') ORDER BY cid;"
    cols_init = run_query(conn, header_query)
    cols = [cols_init[i][0] for i in range(len(cols_init))]
    # get the records of the table
    content_query = "Select * from " + table
    data = run_query(conn, content_query)
    df = pd.DataFrame(data, columns = cols)
    return df

def jaccard_similarity(set1, set2):
    intersection_size = len(set1.intersection(set2))
    union_size = len(set1.union(set2))
    return intersection_size / union_size if union_size != 0 else 0.0


# select the file of the database
database = "/content/Assignment_1.db"

# create a database connection
conn = create_connection(database)

# queries from task 2
query = [
    '''SELECT  AC_ID, customer_ID, balance, branch_Name, street, building_number FROM account ac LEFT JOIN branch br ON ac.branch_ID = br.branch_ID; ''',
    '''SELECT loan.branch_ID, branch_Name, AVG (amount) FROM loan JOIN branch ON loan.branch_ID = branch.branch_ID GROUP BY loan.branch_ID;''',
    '''SELECT customer_ID, balance FROM account WHERE balance = (Select max(balance) From account )'''
]
with conn:
    for q in range(len(query)):
        data = run_query(conn, query[q])
        print(f"Query {q + 1}: {data} \n")

# read customer table and convert it to a dataframe
with conn:
    inst_tab = 'customer'
    df_customer = convert_db_table_to_DF(conn, inst_tab)

display(df_customer)

# Create a list of every single row in df_customer.
records = [df_customer.iloc[row] for row in range(len(df_customer))]

for i in range(len(records)):
    r1 = set(records[i])
    # loop over the columns of the second table
    r2 = set()

    for j in range(i+1,len(records)):
        r2 = set(records[j])
        JSim = jaccard_similarity(r1, r2)

        # Display similarities that score more than 0.7.
        if JSim > 0.7 :
          print (f"Jacard similarity between Customer with IDs {records[i][0]} and {records[j][0]} = {round(JSim, 2)}")




Query 1: [('NL95INGB0123456789', '9876543210', 1000120, 'ING Bank Rotterdam', 'Neude 1', 10002), ('NL14INGB0455679123', '9876543211', 54321.75, 'ING Bank Utrecht', 'Vredenburg 40', 10003), ('NL31INGB0789011115', '1234876952', 987654.32, 'ING Bank Amsterdam', 'Oudegracht 221', 10001), ('NL89INGB0980004321', '7890654321', 12345678.99, 'ING Bank The Hague', 'Mariaplaats 45', 10004), ('NL04INGB0654000098', '4567123980', 7500.5, 'ING Bank Eindhoven', 'Lange Nieuwstraat 24', 10005), ('NL39INGB7212004321', '7881257321', 2678678.69, 'ING Bank The Hague', 'Mariaplaats 45', 10004), ('NL12INGB8912105612', '6891272722', 698175.01, 'ING Bank Amsterdam', 'Oudegracht 221', 10001)] 

Query 2: [('B001', 'ING Bank Amsterdam', 65000.0), ('B002', 'ING Bank Rotterdam', 1000000.0), ('B004', 'ING Bank The Hague', 450000.0), ('B005', 'ING Bank Eindhoven', 115000.0)] 

Query 3: [('7890654321', 12345678.99)] 



,customer_ID,street,number,city,country,name
0,9876543210,Prinsengracht,40,Amsterdam,Netherlands,Jansen
1,9876543211,Prinsengracht,40,Amsterdam,Netherlands,Jansen
2,1234876952,Hauptstraße,88,Stuttgart,Germany,Müller
3,7890654321,Keizerstraat,12,Groningen,Netherlands,Visser
4,4567123980,Buchenweg,27,Lausanne,Switzerland,Wagner
5,7881257321,Molenweg,8,Maastricht,Netherlands,Dijk
6,6891272722,Friedrichstraße,115,Dortmund,Germany,Fischer


Jacard similarity between Customer with IDs 9876543210 and 9876543211 = 0.71
